<a href="https://colab.research.google.com/github/SaahilArora/DataScienceTutorial/blob/master/spam_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [0]:
df = pd.read_csv('./sample_data/spam.csv', encoding='latin1')

In [6]:
df.head(2)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [0]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
stop_words = set(stopwords.words('english')) 
filtered_sentence = []
word_tokens = ''

In [0]:
def clean_text(x):
    word_tokens = word_tokenize(x)
    filtered_sentence = ' '.join(w.lower() for w in word_tokens if w.isalpha() if not w in stop_words)
    return filtered_sentence

In [0]:
df['v2'] = df['v2'].apply(lambda x: x.lower())

In [0]:
df['v2'] = df['v2'].apply(clean_text)

In [35]:
df.head(2)

,v2,Spam
0,go jurong point available bugis n great world ...,0
1,ok lar joking wif u oni,0


In [0]:
df['Spam']=pd.get_dummies(df['v1'], drop_first=True)

In [0]:
df=df.drop('v1', axis=1)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 2000)

In [0]:
X = df['v2'].values

In [63]:
X

array(['go jurong point available bugis n great world la e buffet cine got amore wat',
       'ok lar joking wif u oni',
       'free entry wkly comp win fa cup final tkts may text fa receive entry question std txt rate c apply',
       ..., 'pity mood suggestions',
       'guy bitching acted like interested buying something else next week gave us free',
       'rofl true name'], dtype=object)

In [0]:
Y = df['Spam'].values

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [64]:
X_train.flatten()

array(['u call', 'tell u headache want use hour sick time',
       'never try alone take weight tear comes ur heart falls ur eyes always remember stupid friend share bslvyl',
       ..., 'heart sory', 'nt joking seriously told', 'work going min'],
      dtype=object)

In [0]:
tf_train = vectorizer.fit_transform(X_train.flatten())
tf_test = vectorizer.transform(X_test.flatten())

In [47]:
print(tf_train.shape)
print(tf_test.shape)

(4179, 2000)
(1393, 2000)


In [0]:
tf_train=tf_train.toarray()
tf_test=tf_test.toarray()

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

In [56]:
model = Sequential()
model.add(Dense(units=2000,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=100,activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(units=250,activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
model.fit(x=tf_train,y=y_train,batch_size = 32, epochs=5,validation_data=(tf_test, y_test), verbose=1,callbacks=[early_stop], shuffle=True)

Epoch 1/5
131/131 [==============================] - 5s 40ms/step - loss: 0.1671 - accuracy: 0.9502 - val_loss: 0.0808 - val_accuracy: 0.9792
Epoch 2/5
131/131 [==============================] - 7s 53ms/step - loss: 0.0230 - accuracy: 0.9940 - val_loss: 0.1452 - val_accuracy: 0.9777
Epoch 3/5
131/131 [==============================] - 8s 59ms/step - loss: 0.0052 - accuracy: 0.9988 - val_loss: 0.1369 - val_accuracy: 0.9792
Epoch 4/5
131/131 [==============================] - 5s 39ms/step - loss: 0.0026 - accuracy: 0.9995 - val_loss: 0.1299 - val_accuracy: 0.9792
Epoch 5/5
131/131 [==============================] - 5s 39ms/step - loss: 0.0026 - accuracy: 0.9995 - val_loss: 0.1611 - val_accuracy: 0.9777


In [0]:
predictions = model.predict_classes(tf_test)

In [0]:
from sklearn.metrics import classification_report,confusion_matrix

In [59]:
print(confusion_matrix(y_test,predictions))

[[1200    2]
 [  29  162]]


In [0]:
text = ["Quick Loans A [redacted] loan for £950 is approved for you if you receive this SMS. 1 min verification & cash in 1 hr at www.[redacted].co.uk to opt out reply stop",
        'Live: Spoke to aviation minister, need time to resume flights, Uddhav says']

df1 = pd.Series(data=text)
df1 = df1.apply(clean_text)

In [79]:
print(df1.values.flatten())

['quick loans a redacted loan approved receive sms min verification cash hr www redacted opt reply stop'
 'live spoke aviation minister need time resume flights uddhav says']


In [0]:
data = vectorizer.transform(df1.values.flatten())

In [81]:
data.shape

(2, 2000)

In [0]:
pred = model.predict_classes(data)

In [84]:
pred

array([[1],
       [0]], dtype=int32)